In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [57]:
traindf = pd.read_csv("classifier_dfs/train.csv")
validatedf = pd.read_csv("classifier_dfs/validate.csv")
testdf = pd.read_csv("classifier_dfs/test.csv")

In [3]:
traindf.head(1)

,Unnamed: 0,climb_id,user_id,star_rating,tick_date,ticked,sugg_aid_grade,sugg_boulder_grade,sugg_ice_grade,sugg_mixed_grade,sugg_rock_grade,sugg_ice_grade_number,sugg_boulder_grade_number,sugg_aid_grade_number,sugg_mixed_grade_number,sugg_rock_grade_number,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,105714776_10004,105714776,10004,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
validatedf.head(1)

,user_id,climb_id,star_rating,Unnamed: 0
0,10101,105752563,2,105752563_10101


In [5]:
testdf.head(1)

,user_id,climb_id,star_rating,Unnamed: 0
0,10101,105754933,4,105754933_10101


In [58]:
traindf = traindf[["user_id","climb_id","star_rating","Unnamed: 0"]]

In [7]:
traindf.head(1)

,user_id,climb_id,star_rating,Unnamed: 0
0,10004,105714776,2,105714776_10004


In [59]:
len(traindf)

557046

###REMOVE THE NEXT THREE CELLS IF ALREADY USING SMALLER DATAFRAMES

In [64]:
%%time

climbRatingCutOff = 75
userRatingCutOff = 75

climbsToKeep = [climb for climb, rating in traindf.groupby('climb_id').count().reset_index()[['climb_id','star_rating']].values if rating > climbRatingCutOff]
mask = [True if climb in climbsToKeep else False for climb in list(traindf.climb_id)]
traindf = traindf[mask]
usersToKeep = [user for user, rating in traindf.groupby('user_id').count().reset_index()[['user_id','star_rating']].values if rating > climbRatingCutOff]
mask = [True if user in usersToKeep else False for user in list(traindf.user_id)]
traindf = traindf[mask]


CPU times: user 1.55 s, sys: 35.7 ms, total: 1.59 s
Wall time: 1.61 s


In [65]:
len(traindf)

25942

In [66]:
mask = [True if climb in climbsToKeep else False for climb in list(validatedf.climb_id)]
validatedf = validatedf[mask]
mask = [True if user in usersToKeep else False for user in list(validatedf.user_id)]
validatedf = validatedf[mask]
mask = [True if climb in climbsToKeep else False for climb in list(testdf.climb_id)]
testdf = testdf[mask]
mask = [True if user in usersToKeep else False for user in list(testdf.user_id)]
testdf = testdf[mask]

In [69]:
climbsdf = pd.read_csv("finalproject-sorted_climbs.csv")

//anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [70]:
climbsdf.head(1)

,Unnamed: 0,climb_id,climb_name,climb_link,fa_year,feet,page_views,pitches,avg_stars,grade,state,latitude,longitude,area_0,area_1,area_2,area_3,area_4,area_5,area_6,area_7,area_8,area_9,"type_TR,",type_aid,type_alpine,type_boulder,type_ice,type_mixed,type_sport,type_trad,guide_aid_grade,guide_boulder_grade,guide_ice_grade,guide_mixed_grade,guide_rock_grade,concensus_aid_grade,concensus_boulder_grade,concensus_ice_grade,concensus_mixed_grade,concensus_rock_grade
0,105714713,105714713,garden-party,http://www.mountainproject.com/v/garden-party/...,NaN,60,1157,1,2.0769,NaN,44.0597,-103.4232,0,/destinations/,/v/south-dakota/105708963,/v/falling-rock/105714240,/v/north-canyon-wall/105714312,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,1,0,0,0,0,5.8,0,0,0,0,5.8-


In [71]:
climbsdf = climbsdf[["climb_id", "pitches", "type_TR,", "type_aid", "type_alpine", "type_boulder", "type_ice", "type_mixed", "type_sport", "type_trad", "concensus_aid_grade", "concensus_boulder_grade", "concensus_ice_grade", "concensus_mixed_grade", "concensus_rock_grade"]]

In [72]:
climbsdf.head(1)

,climb_id,pitches,"type_TR,",type_aid,type_alpine,type_boulder,type_ice,type_mixed,type_sport,type_trad,concensus_aid_grade,concensus_boulder_grade,concensus_ice_grade,concensus_mixed_grade,concensus_rock_grade
0,105714713,1,1,0,0,0,0,0,0,1,0,0,0,0,5.8-


In [73]:
climbsdf.columns = ["climb_id", "pitches", "tr", "aid", "alpine", "boulder", "ice", "mixed", "sport", "trad", "aid_grade", "boulder_grade", "ice_grade", "mixed_grade", "rock_grade"]

In [74]:
climbsdf["aid_grade_num"] = [int(x[1]) + (0.3 if len(x) > 2 and x[2] == "+" else 0) - (0.3 if len(x) > 2 and x[2] == "-" else 0) if x != "0" else float('nan') for x in climbsdf.aid_grade ]

In [75]:
np.unique([type(x) for x in climbsdf.aid_grade_num])

array([<type 'numpy.float64'>], dtype=object)

In [76]:
climbsdf["boulder_grade_num"] = [int(x[1]) + (0.3 if len(x) > 2 and x[2] == "+" else 0) - (0.3 if len(x) > 2 and x[2] == "-" else 0) if x != "0" else float('nan') for x in climbsdf.boulder_grade ]

In [77]:
climbsdf["ice_grade_num"] = [int(x[2]) + (0.3 if len(x) > 3 and x[3] == "+" else 0) - (0.3 if len(x) > 3 and x[3] == "-" else 0) if x != "0" else float('nan') for x in climbsdf.ice_grade]

In [78]:
np.unique(list(climbsdf.mixed_grade))

array(['0', 'M1', 'M1-', 'M11', 'M2', 'M2-', 'M3', 'M4', 'M4+', 'M4-',
       'M5', 'M5+', 'M6', 'M6+', 'M6-', 'M7', 'M7+', 'M8', 'M8+'], 
      dtype='|S3')

In [79]:
climbsdf["mixed_grade_num"] = [(int(x[1:]) if len(x) > 2 and x[2] != "+" and x[2] != "-" else int(x[1]))+ (0.3 if len(x) > 2 and x[2] == "+" else 0) - (0.3 if len(x) > 2 and x[2] == "-" else 0) if x != "0" else float('nan') for x in climbsdf.mixed_grade]

In [80]:
np.unique(climbsdf.rock_grade)

array(['0', '5-6', '5.0', '5.1', '5.10', '5.10+', '5.10-', '5.10a',
       '5.10a/b', '5.10b', '5.10b/c', '5.10c', '5.10c/d', '5.10d', '5.11',
       '5.11+', '5.11-', '5.11a', '5.11a/b', '5.11b', '5.11b/c', '5.11c',
       '5.11c/d', '5.11d', '5.12', '5.12-', '5.12a', '5.12a/b', '5.13-',
       '5.13b', '5.3', '5.4', '5.5', '5.6', '5.7', '5.7+', '5.8', '5.8+',
       '5.8-', '5.9', '5.9+', '5.9-'], dtype=object)

In [81]:
[1 for x in climbsdf.rock_grade if x == '5-6']

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [82]:
climbsdf["rock_grade_num"] = [(float(x[2:x.find("+")]) + 0.3 if x.find("+") != -1 else
                               float(x[2:x.find("-")]) -0.3 if x.find("-") != -1 else
                               float(x[2:x.find("a/b")]) - 0.25 if x.find("a/b") != -1 else
                               float(x[2:x.find("b/c")]) if x.find("b/c") != -1 else
                               float(x[2:x.find("c/d")]) + 0.25 if x.find("c/d") != -1 else
                               float(x[2:x.find("a")]) - 0.375 if x.find("a") != -1 else
                               float(x[2:x.find("b")]) - 0.125 if x.find("b") != -1 else
                               float(x[2:x.find("c")]) + 0.125 if x.find("c") != -1 else
                               float(x[2:x.find("d")]) + 0.375 if x.find("d") != -1 else
                               float(x[2:]))
                              if x != '0' and x != '5-6' else float('nan') for x in climbsdf.rock_grade 
                             ]

In [83]:
climbsdf.head(1)

,climb_id,pitches,tr,aid,alpine,boulder,ice,mixed,sport,trad,aid_grade,boulder_grade,ice_grade,mixed_grade,rock_grade,aid_grade_num,boulder_grade_num,ice_grade_num,mixed_grade_num,rock_grade_num
0,105714713,1,1,0,0,0,0,0,0,1,0,0,0,0,5.8-,NaN,NaN,NaN,NaN,7.7


In [84]:
aid_mean = np.mean(climbsdf.aid_grade_num)
aid_std = np.std(climbsdf.aid_grade_num)
boulder_mean = np.mean(climbsdf.boulder_grade_num)
boulder_std = np.std(climbsdf.boulder_grade_num)
ice_mean = np.mean(climbsdf.ice_grade_num)
ice_std = np.std(climbsdf.ice_grade_num)
mixed_mean = np.mean(climbsdf.mixed_grade_num)
mixed_std = np.std(climbsdf.mixed_grade_num)
rock_mean = np.mean(climbsdf.rock_grade_num)
rock_std = np.std(climbsdf.rock_grade_num)

In [85]:
climbsdf["aid_grade_norm"] = (climbsdf.aid_grade_num - aid_mean)/aid_std
climbsdf["boulder_grade_norm"] = (climbsdf.boulder_grade_num - boulder_mean)/boulder_std
climbsdf["ice_grade_norm"] = (climbsdf.ice_grade_num - ice_mean)/ice_std
climbsdf["mixed_grade_norm"] = (climbsdf.mixed_grade_num - mixed_mean)/mixed_std
climbsdf["rock_grade_norm"] = (climbsdf.rock_grade_num - rock_mean)/rock_std

In [86]:
climbsdf["norm_grade"] = climbsdf[["aid_grade_norm", "boulder_grade_norm", "ice_grade_norm", "mixed_grade_norm", "rock_grade_norm"]].apply(np.nanmean, axis=1)

//anaconda/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


In [87]:
climbsdf["norm_grade"] = [0 if np.isnan(x) else x for x in climbsdf.norm_grade ]

In [88]:
climbsdf.head(1)

,climb_id,pitches,tr,aid,alpine,boulder,ice,mixed,sport,trad,aid_grade,boulder_grade,ice_grade,mixed_grade,rock_grade,aid_grade_num,boulder_grade_num,ice_grade_num,mixed_grade_num,rock_grade_num,aid_grade_norm,boulder_grade_norm,ice_grade_norm,mixed_grade_norm,rock_grade_norm,norm_grade
0,105714713,1,1,0,0,0,0,0,0,1,0,0,0,0,5.8-,NaN,NaN,NaN,NaN,7.7,NaN,NaN,NaN,NaN,-1.319268,-1.319268


In [89]:
climbsdf = climbsdf[["climb_id", "pitches", "tr", "aid", "alpine", "boulder", "ice", "mixed", "sport", "trad", "norm_grade"]]

In [90]:
climbsdf.head(1)

,climb_id,pitches,tr,aid,alpine,boulder,ice,mixed,sport,trad,norm_grade
0,105714713,1,1,0,0,0,0,0,0,1,-1.319268


In [91]:
pitches_mean = np.mean(climbsdf.pitches)
pitches_std = np.std(climbsdf.pitches)
climbsdf["pitches_norm"] = (climbsdf.pitches - pitches_mean)/pitches_std

In [92]:
climbsdf["pitches_norm"] = [0 if np.isnan(x) else x for x in climbsdf.pitches_norm]

In [93]:
climbsdf.head(1)

,climb_id,pitches,tr,aid,alpine,boulder,ice,mixed,sport,trad,norm_grade,pitches_norm
0,105714713,1,1,0,0,0,0,0,0,1,-1.319268,-0.307332


In [94]:
climbsdf = climbsdf[[x for x in list(climbsdf.columns) if x != "pitches"]]

In [95]:
climbsdict = {int(x[0]) : {'tr': x[1], 'aid': x[2], 'alpine': x[3], 'boulder': x[4], 'ice': x[5], 'mixed': x[6], 
                           'sport': x[7], 'trad': x[8], 'grade': x[9], 'pitches': x[10]} for x in climbsdf.values}

In [96]:
userlist = np.unique(list(traindf.user_id) + list(validatedf.user_id) + list(testdf.user_id))
userdict = dict(zip(userlist,range(len(userlist))))

In [97]:
climbslist = np.unique(list(traindf.climb_id) + list(validatedf.climb_id) + list(testdf.climb_id))

In [98]:
[x for x in climbslist if x not in climbsdict.keys()]

[]

In [99]:
climbsdict2 = dict(zip(climbslist,range(len(climbslist))))

In [100]:
def make_feature_matrix(indf):
    num_features = len(climbsdict.values()[0])
    featurematrix = np.zeros(shape=(len(indf), len(userdict) + len(climbsdict2) + len(climbsdict2) * num_features))
    inarray = indf[["user_id", "climb_id", "star_rating"]].values
    for i in range(len(inarray)):
        featurematrix[i, userdict[inarray[i,0]]] = 1
        featurematrix[i, len(userdict) + climbsdict2[inarray[i,1]]] = 1
        for i in range(num_features):
            featurematrix[i, (len(userdict) + len(climbsdict2) + climbsdict2[inarray[i,1]] * num_features) + i] = climbsdict[inarray[i,1]].values()[i]
    return (featurematrix, indf.star_rating.values)

###UNCOMMENT THE LAST TWO LINES ONCE USING BETTER DONE DATAFRAMES

In [102]:
%%time

train_fm, train_stars = make_feature_matrix(traindf)
#validate_fm, validate_stars = make_feature_matrix(validatedf)
#test_fm, test_stars = make_feature_matrix(testdf)

CPU times: user 365 ms, sys: 86.9 ms, total: 452 ms
Wall time: 454 ms


In [ ]:
print "Matrix Shapes:"
print 
print "train_fm", train_fm.shape
print "train_stars", train_stars.shape
print "validate_fm", validate_fm.shape
print "validate_stars", validate_stars.shape
print "test_fm", test_fm.shape
print "test_stars", test_stars.shape

In [103]:
from sklearn.linear_model import Ridge

In [104]:
%%time

vdict={}
rdict={}
for alpha in [0.1]: #[0.01, 0.1, 1, 10, 100, 1000]:
    regression = Ridge(alpha=alpha).fit(train_fm, train_stars)
    rdict[alpha] = regression
    predictions_actual_zip = zip(list(regression.predict(validate_fm)),list(validate_stars))
    se_sum = 0
    for (a,b) in predictions_actual_zip:
        se_sum = se_sum + (a - b)**2
    vdict[alpha] = np.sqrt(se_sum / len(predictions_actual_zip))

CPU times: user 1min 12s, sys: 1.5 s, total: 1min 14s
Wall time: 23.1 s
